In [ ]:
# TODO: test taht ps3.get_pdf_text(asset_url) is working as expected.


In [ ]:
# !pip uninstall -y foresight
#!pip install git+https://github.com/SAGE-3/next.git@dev#subdirectory=foresight

In [ ]:
# !pip install langchain_openai
# !pip install langchain

In [1]:
PROMPTS_DIR = "src/prompts/"


In [ ]:
# def remove_keys_from_dict(app_dict, keys_to_remove):
#     def _remove_keys_from_dict(d, keys_to_remove):
#         if isinstance(d, dict):
#             for key in list(d.keys()):  # Use list() to avoid 'dictionary changed size during iteration' error
#                 if key in keys_to_remove:
#                     del d[key]
#                 else:
#                     _remove_keys_from_dict(d[key], keys_to_remove)
#         elif isinstance(d, list):
#             for item in d:
#                 _remove_keys_from_dict(item, keys_to_remove)

#     # Create a deep copy of the input dictionary
#     app_dict_copy = copy.deepcopy(app_dict)
#     # Remove keys from the copied dictionary
#     _remove_keys_from_dict(app_dict_copy, keys_to_remove)
#     return app_dict_copy


In [2]:
import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")
# from src.config import config as conf, prod_type
import copy
from foresight.Sage3Sugar.pysage3 import PySage3
from src.prompts.prompt_loader import PromptLoader 
from langchain_openai import ChatOpenAI
keys_to_remove = ['_id', '_createdAt', '_createdBy', '_updatedAt', '_updatedBy',
    'touched', 'executeInfo', 'path', 'raised', 
    'title', 'roomId', 'boardId', 'lock', 'dragging', 'pinned', 'rotation'
]


In [3]:
prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}
room_id = 'd6640933-7b9f-4536-a263-02a514ff092a'
board_id = '02fc98df-b4f1-470c-8516-7221c3095de7'
# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'

In [4]:
ps3 = PySage3(conf, prod_type)
cb = ps3.rooms['d6640933-7b9f-4536-a263-02a514ff092a'].boards['02fc98df-b4f1-470c-8516-7221c3095de7']

Configuring ps3 client ... 
Completed configuring Sage3 Client


In [5]:
ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)

{'b18bd2d2-0840-40c9-bfc8-5970e5558c6c': {'data': {'position': {'x': 1500286,
    'y': 1500222,
    'z': 0},
   'size': {'width': 400, 'height': 420, 'depth': 0},
   'type': 'Stickie',
   'state': {'text': 'test yellow', 'fontSize': 36, 'color': 'yellow'}},
  'tags': ['first', 'yellow']},
 '96fa5557-16ee-4597-8708-bca17c6c7c02': {'data': {'position': {'x': 1500761,
    'y': 1500217,
    'z': 0},
   'size': {'width': 400, 'height': 420, 'depth': 0},
   'type': 'Stickie',
   'state': {'text': 'test red', 'fontSize': 36, 'color': 'red'}},
  'tags': ['red']}}

In [6]:
llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
#llm_4o.invoke("hello")

In [7]:
example_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "few_shot", "reframe_select_apps")
q = 'Can you delete all the red stickies with a greeting or hello'
# prompt_text = example_prompt.format(query=q)
(example_prompt | llm_4o).invoke({"query":q})

AIMessage(content='```json\n{\n  "updated_query": "Select the red stickies with a greeting or hello"\n}\n```', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 302, 'total_tokens': 325}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-1060e0b0-3c61-4232-884b-a625e8e08f0c-0')

In [11]:
example_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "few_shot", "select_apps")

q = 'Can you delete all the red stickies with a greeting'
all_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
prompt_text = example_prompt.format(originalItems=all_apps, query=q)
print(prompt_text)


Given the list of items below, identify the app_ids (UUID4) of the items that match the user's query defined below. If none of the items match the query, return an empty list.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"uuids": {"description": "A valid Python list of UUIDs selected", "items": {"format": "uuid", "type": "string"}, "title": "Uuids", "type": "array"}}, "required": ["uuids"]}
```

Example:
originalItems: [{'app_id': 'ba88e0df-d885-4ce3-ad74-006fbba69b43',
  'data': {'position': {'x': 1502426, 'y': 1500697, 'z': 0},
   'size': {'width': 400, 'height': 517, 

In [13]:
(example_prompt | llm_4o).invoke({"originalItems":all_apps, "query":q})

AIMessage(content='```json\n{\n  "uuids": ["96fa5557-16ee-4597-8708-bca17c6c7c02"]\n}\n```', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 1080, 'total_tokens': 1115}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a021f0f-16ab-483b-a104-d34b39121e83-0')